<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="left"src="images/etcbc4easy-small.png"/></a>
<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="left" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="right" src="images/VU-ETCBC-xsmall.png"/></a>

# Semantic Vectors

Joint work of Harald Baayen en Martijn Naaijer and Dirk Roorda.



In [1]:
import sys,os, re
import collections
from IPython.display import HTML, display_pretty, display_html

import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare

fabric = LafFabric()

  0.00s This is LAF-Fabric 4.5.12
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: https://shebanq.ancient-data.org/static/docs/featuredoc/texts/welcome.html



In [2]:
source = 'etcbc'
version = '4b'

In [6]:
API = fabric.load(source+version, 'lexicon', 'semantic', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        otype
        language lex phono gloss
        sp
        book chapter verse number
        freq_lex
    ''',''),
    "prepare": prepare,
    "primary": False,
}, verbose='NORMAL')
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s USING main  DATA COMPILED AT: 2015-11-02T15-08-56
  0.00s USING annox DATA COMPILED AT: 2016-01-27T19-01-17
  1.84s INFO: LOADING PREPARED data: please wait ... 
  1.85s ETCBC reference: http://laf-fabric.readthedocs.org/en/latest/texts/ETCBC-reference.html
  3.13s INFO: LOADED PREPARED data
  3.13s INFO: DATA LOADED FROM SOURCE etcbc4b AND ANNOX lexicon FOR TASK semantic AT 2016-02-23T06-58-31


In [4]:
genre_specs = dict(
    prose={
        'Genesis', 'Exodus', 'Leviticus', 'Numeri', 'Deuteronomium', 
        'Josua', 'Judices', 'Samuel_I', 'Samuel_II', 'Reges_I', 'Reges_II', 
        'Jona', 'Ruth',   'Esther', 'Daniel', 'Esra', 'Nehemia', 'Chronica_I', 'Chronica_II',
    },
    prophecy={
        'Jesaia', 'Jeremia', 'Ezechiel', 'Hosea', 'Joel', 
        'Obadia', 'Micha', 'Zephania', 'Haggai', 'Sacharia', 'Maleachi', 
        'Amos', 'Nahum', 'Habakuk',
    },
    poetry={
        'Canticum','Proverbia','Ecclesiastes', 'Threni', 'Psalmi', 'Iob',
    },
)

genre_books = collections.defaultdict(set)
genre_sent = {}

for b in F.otype.s('book'):
    book_name = F.book.v(b)
    for g in genre_specs:
        if book_name in genre_specs[g]:
            genre_books[g].add(b)
            for s in L.d('sentence', b):
                genre_sent[s] = g

In [9]:
msg('List of adjectives')
adjvs = set()
for w in F.otype.s('word'):
    if F.sp.v(w) == 'adjv':
        adjvs.add((F.lex.v(w), F.gloss.v(w)))
av = open('adjvs.txt', 'w')
for a in sorted(adjvs): av.write('{}\t{}\n'.format(*a))
av.close()
msg('Done {} adjectives'.format(len(adjvs)))

 1m 40s List of adjectives
 1m 42s Done 606 adjectives


In [10]:
emo_adj_lines = '''
<YJB/	sad
<Z===/	strong
<ZWZ=/	powerful
>BJR/	strong
>BJR=/	strong
>BL===/	mourning
>CM=/	guilty
>DJR/	mighty
>GM=/	grieved
>JM/	frightful
>JMTNJ/	terrible
>KZR/	cruel
>KZRJ/	cruel
>MJY/	strong
>PJQ=/	strong
>WJL/	foolish
>WLJ/	awkward
BLH/	worn out
C<RWR/	horrible
C<RWRJ/	horrible
C>NN/	at ease
CLH/	carefree
CLJV/	mighty
DK/	oppressed
DWH/	unwell
DWJ=/	ill
FGJ>/	exalted
FMX/	joyful
G>/	haughty
G>H/	haughty
J<P/	weary
JG</	weary
JGJ<=/	weary
JGWR/	fearing
JHJR/	haughty
JR>/	afraid
KSJL/	insolent
MGN=/	insolent
MR/	bitter
MRD=/	rebellious
MRJRJ/	bitter
MRR/	bitter
MZH/	hungry
N<JM/	pleasant
N>WH/	lovely
NBL/	stupid
NDJB/	willing
NQJ/	innocent
PSX=/	lame
PTLTL/	queer
QCB=/	attentive
QCB==/	attentive
QN>/	jealous
QNW>/	jealous
R<B=/	hungry
RC</	guilty
RGZ=/	excited
RXMNJ/	compassionate
RXWM/	compassionate
S<P/	despicable
SKL/	foolish
SLX/	forgiving
SMR/	bristly
SR/	sullen
SRB/	obstinate
TQJP/	mighty
TQJP/	strong
WZR/	guilty
XNP/	alienated
XNWN/	gracious
XPY=/	delighting
XRD/	trembling
XSJD/	loyal
XSN/	strong
XT=/	terror-filled
XZQ/	strong
XZQ==/	strong
YM>/	thirsty
YNW</	humble
Z<P=/	raging
ZD/	insolent
'''.strip().split('\n')
emo_adj = {x.split('\t')[0] for x in emo_adj_lines }

In [12]:
print(len(emo_adj))

80


In [5]:
msg('Counting hapaxes')
hapax = set()
for w in F.otype.s('word'):
    if F.freq_lex.v(w) == '1': hapax.add(w)
msg('{} hapaxes'.format(len(hapax)))

    16s Counting hapaxes
    18s 3076 hapaxes


In [9]:
msg('Reading sentences')
rows = []
vocabulaire = set()
for (i, s) in enumerate(F.otype.s('sentence')):
    lexemes = set()
    wnodes = set()
    for w in L.d('word', s):
        lex = F.lex.v(w).replace('_', '~')
        if lex not in lexemes:
            if w not in hapax:
                lexemes.add(lex)
                wnodes.add(w)
    vocabulaire |= lexemes
    for w in sorted(wnodes):
        rows.append((i, s, w, wnodes - {w}))
msg('Done. {} rows; {} lexemes (non-hapax)'.format(len(rows), len(vocabulaire)))

 1m 07s Reading sentences
 1m 10s Done. 374421 rows; 5889 lexemes (non-hapax)


In [14]:
msg('Producing cues')
rowfile = {}
gcount = collections.Counter()

for g in genre_specs:
    rowfile[g] = open('rows_{}.txt'.format(g), 'w')
    rowfile[g].write('i\tword\tgloss\tcues\tfreq\n')
all_cues = set()

for (i, s, w, cues) in rows:
    cues_neat = '_'.join(sorted(F.lex.v(w).replace('_', '~') for w in cues))
    all_cues.add(cues_neat)
    g = genre_sent[s]
    rowfile[g].write('{}\t{}\t{}\t{}\t{}\n'.format(i+1, F.lex.v(w).replace('_', '~'), F.gloss.v(w), cues_neat, 1))
    gcount[g] += 1

for g in genre_specs:
    rowfile[g].close()
msg('Done: {} rows, {} different cues'.format(len(rows), len(all_cues)))
for g in gcount:
    print('{:<8}: {:>5} rows'.format(g, gcount[g]))

 4m 09s Producing cues
 4m 14s Done: 374421 rows, 314333 different cues


poetry  : 49764 rows
prophecy: 86871 rows
prose   : 237786 rows
